<a href="https://colab.research.google.com/github/Wattsy2020/DeepLearningA1/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP5329 Assignment 1

Team Member
- Mirope Yuhao Hu, SID: 470139936
- Sean Hongbo Du,	SID: 500635346
- Liam Watts, SID: 510562348


### Liam Watts: 1, 2, 6 - More than 1 hidden layer, ReLU Activation, Softmax output layer and cross entroppy loss
### Mirope: 4, 3, 8 - Momentum in SGD, weight decay, batch normalization
### Sean: 5, 7, 0 - dropout, mini-batch training, data preprocess

## How to run the code

## All necessary Imports

__NOT__ allowed:
1. use Deep Learning frameworks (e.g. PyTorch, Tensorflow, Caffe, and KERAS)
2. any kinds of auto-grad tools (e.g. autograd)
3. sklearn

In [1]:
import numpy as np

## Data Loading/Preprocessing

In [1]:
def oneHot_encode():
    pass

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load data -> subject to change for the final version
dir_name = '/content/drive/MyDrive/Colab Notebooks/COMP5329/Ass1/Assignment1-Dataset/'

test_data = np.load(dir_name + 'test_data.npy')
test_label = np.load(dir_name + 'test_label.npy')
train_data = np.load(dir_name + 'train_data.npy')
train_label = np.load(dir_name + 'train_label.npy')

print('Shape of original train data is:\t', train_data.shape,
      '\nShape of original train label is:\t', train_label.shape,
      '\nShape of original test data is:\t\t', test_data.shape,
      '\nShape of original test label is:\t', test_label.shape )

# print('\nAll labels are:\t\t', list(np.unique(test_label)))

# # preprocessing: normalization might be unnecessary
# # test_x = Normalizer().fit_transform(test_data)
# # train_x = Normalizer().fit_transform(train_data)
# test_x = test_data
# train_x = train_data
# test_y = OneHotEncoder().fit_transform(test_label).toarray()
# train_y = OneHotEncoder().fit_transform(train_label).toarray()

# print('\nShape of transformed train label is:\t', train_y.shape,
#       '\nShape of transformed train label is:\t', test_y.shape )

Shape of original train data is:	 (50000, 128) 
Shape of original train label is:	 (50000, 1) 
Shape of original test data is:		 (10000, 128) 
Shape of original test label is:	 (10000, 1)


## Algorithms / Structures

__Requirements__
1. More than one hidden layer
2. ReLU activation
3. Weight decay
4. Momentum in SGD
5. Dropout
6. Softmax and cross-entropy 
7. Mini-batch training
8. Batch Normalization

In this part, try to implement each layer/optimiser/function as modules

In [5]:
# Liam Watts: 1, 2, 6 - activation, layer, softmax
# Mirope: 4, 3, 8 - Momentum in SGD, weight decay, batch normalization
# Sean: 5, 7, 0 - dropout, mini-batch training, data preprocess

In [2]:
# Define functions required for the Multi Layer Perceptron
def he_uniform(input_size=1, output_size=1):
    '''outputs a matrix of size (output_size, input_size) with random initialization'''
    r = np.sqrt(6/input_size)
    return np.random.uniform(-r, r, size=(output_size, input_size))

def relu(matrix):
    '''return the elementwise ReLU activation of the matrix'''
    return np.maximum(matrix, 0)

def drelu(matrix):
    '''
    parameters:
        matrix: the relu activations
    
    outputs:
        the derivative for relu on every element of the matrix
    '''
    return np.minimum(np.maximum(matrix, 0), 1) # this defines the derivative of relu at 0 as 0 to tf.nn.relu()'s standard (which ensures a sparser gradient matrix)

# define the sigmoid function for testing, it's easier to implement backprop on sigmoid than softmax
def sigmoid(matrix):
    '''return sigmoid on every element of the matrix'''
    return 1/(1 + np.power(np.e, -matrix))

def dlogsigmoid(matrix):
    '''
    parameters:
        matrix: the sigmoid activations
    
    outputs:
        the derivative for log sigmoid on every element of the matrix
    '''
    return (1 - matrix) # dsigmoid = sigmoid(1 - sigmoid)  so dlog(sigmoid) = sigmoid(1 - sigmoid)/sigmoid = (1 - sigmoid)

def softmax(matrix):
    '''return the softmax activation of a matrix (each row vector has softmax applied to it)'''
    matrix = np.power(np.e, matrix)
    total = np.sum(matrix, axis=1).reshape(matrix.shape[0], 1)
    return matrix/total # here we broadcast total (a vector) to divide each row in the matrix, broadcasting is frequently used to vectorize our code

# it is faster to calculate the derivative of log(softmax(vector)), and because we use cross entropy as the loss function
# the only derivative we need to calculate is this log(softmax(vector))
def dlogsoftmax(vector):
    '''
    parameters:
        vector: the softmax activations
    
    outputs:
        the jacobian, a matrix of derivatives of log(vector[i]) wrt vector[j]
    '''
    pass # to implement (examine this https://stackoverflow.com/questions/35304393/trying-to-understand-code-that-computes-the-gradient-wrt-to-the-input-for-logsof)

In [14]:
class MLP:
    ''' 
    Implements a Multi Layer Perceptron, including both the feed-forward phase and backward propagation
    The output activation must be softmax, and the loss function cross entropy

    MLP.__init__()
        parameters:
            hidden: an array with the number of neurons for each layer
        
        initializes weight matrices with shape (output_size, input_size) to match Pytorch's standard
    '''
    def __init__(self, hidden=[1], input_size=1, output_size=1, learn_rate=0.01, initializer=he_uniform):
        '''initialize the weight and bias matrices for each layer, along with other hyperparameters'''
        self.weights = []
        self.biases = []
        for input_size_i, output_size_i in zip([input_size] + hidden, hidden + [output_size]):
            self.weights.append(initializer(input_size=input_size_i, output_size=output_size_i))
            self.biases.append(np.zeros((output_size_i, 1)))

        self.learn_rate=learn_rate

    def forward(self, X):
        '''
        calculate output of the network, storing the hidden states in self.activations (activations[0] is the feature input)
        
        parameters:
            X: the input matrix of shape (batch_size, input_size)

        returns:
            yhat: the prediction matrix of shape (batch_size, output_size)
        '''
        self.activations = [X]
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            if i == len(self.weights) - 1: # use sigmoid activation in the last layer. TODO: change to softmax
                activation = self.forward_one_layer(self.activations[i], w, b, activation=sigmoid)
                #print("activation: ", activation[0, :])
            else:
                activation = self.forward_one_layer(self.activations[i], w, b, activation=relu)
            self.activations.append(activation)
        return self.activations[-1]

    def forward_one_layer(self, inputs, weights, biases, activation=relu):
        '''
        compute the output of a layer for certain inputs, return the output
        
        we follow the pytorch standard, so shapes are as follows:
        inputs: a matrix of shape (batch_size, feature_size)
        weights: a matrix of shape (output_size, feature_size)
        biases: a matrix of shape (output_size, 1)
        returns: a matrix of shape (batch_size, output_size)
        '''
        linear = np.dot(inputs, weights.T) + biases.T
        return activation(linear)

    def backprop(self, y, yhat):
        '''given the true outputs y and predictions yhat, perform one step of gradient descent'''
        assert self.activations != [], "forward propagate first before back propagation" # we need the hidden states
        # get the derivative of cross-entropy loss for the output
        dloss = yhat - y # TODO: use softmax log loss instead

        # do backprop layer by layer, passing to each layer the derivative of loss w.r.t that layer's outputs
        for layer in range(len(self.weights) - 1, -1, -1):
            dloss = self.backprop_one_layer(dloss, layer)
        self.activations = [] # reset the activations to avoid using them twice for the same backprop

    def backprop_one_layer(self, dloss, i):
        '''
        backpropagate one layer of the model
        parameters:
            i: the index of the layer to propagate in self.weights and self.biases
            dloss: loss of the output (pre relu) of this layer, shape of (batch_size, layer_output_size)
        
        updates the weights and biases by subtracting the gradient

        returns:
            dloss: loss of the output (pre relu) of the previous layer (layer i-1), so long as layer i-1 exists (i.e. i-1 >= 0)
        '''
        assert dloss is not None, "Output deltas not provided"
        
        # outline dimensions
        batch_size = dloss.shape[0]
        layer_output_size = dloss.shape[1]
        layer_input_size = self.activations[i].shape[1]
        
        # calculate gradients
        activation = self.activations[i].reshape(batch_size, layer_input_size) # use reshapes to assert the shape is correct, and indicate shape
        dw = np.dot(dloss.T, activation).reshape(layer_output_size, layer_input_size)
                                         # the i,j entry of dw represents the dot product of:
                                         # the ith row of dloss.T i.e. the derivative of losses for neuron i over all samples
                                         # the jth column of activation i.e. the input to weight j over all samples
                                         # so overall: it is (dloss for neuron i * input for weight j of neuron i), giving us the derivative of loss w.r.t weight i,j
        db = np.sum(dloss.T, axis=1).reshape(layer_output_size, 1)
                                     # output = X*W.T + B so doutput w.r.t db is just 1, hence we only need to sum the gradients for each bias
        
        # calculate gradient of loss w.r.t output of previous layer (if there is a previous layer)
        if i != 0:
            dloss_prev = np.dot(dloss, self.weights[i]).reshape(batch_size, layer_input_size)
                                          # the i,j entry of dloss_prev represents the dot product of
                                          # the ith row of dloss i.e. the dlosses for the ith sample for each of this layers outputs
                                          # the jth column of W i.e. the jth weight for all output neurons in this layer
                                          # overall: the sum of (dloss of output neuron k * weight j (the weight that multiplies input j) of output neuron k for sample i)
                                          # which is the derivative of the loss w.r.t input j for this sample as output = X*W.T + B
            dloss_prev = dloss_prev * drelu(activation) # multiply by the derivatives of the previous layers activations
        else:
            dloss_prev = None

        # update gradients
        self.weights[i] -= (self.learn_rate/batch_size)*dw
        self.biases[i] -= (self.learn_rate/batch_size)*db

        return dloss_prev

In [17]:
# test sigmoid MLP training
# create a dataset for training
X = np.random.rand(10000, 4)
#y = (np.sum(X, axis=1) > 2) # the task is to identify if the sum of 4 numbers is greater than 2
y = X[:, 0] > 0.5
y = y.reshape(10000, 1)
np.concatenate([X, y], axis=1)

array([[0.70971506, 0.40725527, 0.16487558, 0.79116544, 1.        ],
       [0.55246974, 0.14443199, 0.52638744, 0.01795876, 1.        ],
       [0.30822828, 0.28431978, 0.91892987, 0.54763407, 0.        ],
       ...,
       [0.68330851, 0.75867031, 0.70576458, 0.79125109, 1.        ],
       [0.6212175 , 0.56333251, 0.73478416, 0.2529044 , 1.        ],
       [0.28774412, 0.23783147, 0.38273565, 0.75358385, 0.        ]])

In [20]:
mlp = MLP(hidden=[100, 50, 25], input_size=4, output_size=1, learn_rate=0.01)

# get accuracy function
def accuracy(mlp, X, y):
    preds = mlp.forward(X) > 0.5
    accuracy = np.mean(preds == y)
    return accuracy

# train model
batch_size = 100
for epoch in range(100):
    # shuffle
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices, :]
    y = y[indices, :]

    for batch in range(int(X.shape[0]/batch_size)):
        batch_x = X[batch*batch_size:(batch+1)*batch_size]
        batch_y = y[batch*batch_size:(batch+1)*batch_size]
        preds = mlp.forward(batch_x)
        mlp.backprop(batch_y, preds)

    if epoch % 10 == 0:
        print("Epoch {}    Accuracy: {}".format(epoch, accuracy(mlp, X, y)))

Epoch 0    Accuracy: 0.6262
Epoch 10    Accuracy: 0.936
Epoch 20    Accuracy: 0.9646
Epoch 30    Accuracy: 0.977
Epoch 40    Accuracy: 0.984
Epoch 50    Accuracy: 0.9886
Epoch 60    Accuracy: 0.9903
Epoch 70    Accuracy: 0.9932
Epoch 80    Accuracy: 0.9948
Epoch 90    Accuracy: 0.994


In [89]:
mlp.weights[-1], mlp.biases[-1]

(array([[15.29173464, -0.27511754, -0.20489533, -0.22305846]]),
 array([[-7.22438466]]))

In [ ]:
mlp = MLP(hidden=[64], input_size=128, output_size=10)
preds = mlp.forward(train_data)
preds, np.sum(preds, axis=1)

(array([[0.03418064, 0.03418064, 0.27683855, ..., 0.03959415, 0.05997034,
         0.28327345],
        [0.07543711, 0.03333939, 0.03333939, ..., 0.03333939, 0.03333939,
         0.1027464 ],
        [0.07403353, 0.07403353, 0.07403353, ..., 0.14066237, 0.07403353,
         0.07403353],
        ...,
        [0.10015951, 0.11925923, 0.08579675, ..., 0.18000399, 0.08579675,
         0.08579675],
        [0.11504859, 0.01624576, 0.01624576, ..., 0.01624576, 0.01624576,
         0.14411416],
        [0.11432698, 0.08168115, 0.08168115, ..., 0.08168115, 0.08168115,
         0.23222384]]), array([1., 1., 1., ..., 1., 1., 1.]))

## Experiments

In [ ]:
# the baseline model
def baseline_model():
	# create model
    model = Sequential()
    model.add(Dense(512, input_dim=128, activation='relu')) # 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1)) # 5
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1)) # 5
    model.add(BatchNormalization()) # 8
    model.add(Dense(10, activation='softmax')) # 1, 6.1
    opt = SGD(lr=0.3, momentum=0.9, decay=0.001) # optimizers: 3, 4
    # 6.2
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

  
# Crossing Validation
def training(model, epoches=20, batch_size=256):
  for i in epoches:
    